### Buiding a chatbot with langgraph

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [3]:
class State(TypedDict):
  messages:Annotated[list,add_messages]
graph_builder=StateGraph(State)


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq
llm=ChatGroq(api_key=GROQ_API_KEY,model="llama-3.1-8b-instant",temperature=0.7)

In [6]:
def chatbot(state:State):#since the input state is of type dict, we are trying to inherit from State
  return {"messages":[llm.invoke(state["messages"])]}

In [ ]:
graph_builder=StateGraph(State)
graph_builder.add_node